<a href="https://colab.research.google.com/github/susmitmishra125/QA_NLP/blob/main/NLP_Q_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/susmitmishra125/QA_NLP

Cloning into 'QA_NLP'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 5), reused 8 (delta 2), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [16]:
!pip install contractions

     |████████████████████████████████| 321 kB 7.3 MB/s 
     |████████████████████████████████| 284 kB 59.6 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85447 sha256=ce4b567f43c6a61d27bf9de1b13807bcac3ae974f9591ed4f113ab2728975c67
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [29]:
# Importing modules
import nltk
nltk.download('punkt') # For tokenizers
from nltk.tokenize import word_tokenize,sent_tokenize
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
import contractions
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# importing dataset
data = open('QA_NLP/passage.txt','r',encoding="utf-8").read()

def preprocessing(rawReadCorpus):
    pattern = "^a-zA-Z0-9_"
    rawReadCorpus = contractions.fix(rawReadCorpus)
    # rawReadCorpus = pattern.sub('', str)
    # rawReadCorpus = rawReadCorpus.replace("’","qqq") # this is to make sure words like wouldn't are in a same token, after tokenisation @ would be replace with ’
    # rawReadCorpus = rawReadCorpus.replace("—"," ") # — is replaced with space
    # double quotes are removed as they prevented the sent_tokenize to separate sentences which ended with ” and are replaced with single " 
    # rawReadCorpus = rawReadCorpus.replace("“"," ")
    # rawReadCorpus = rawReadCorpus.replace("”"," ")

    text_sent = sent_tokenize(rawReadCorpus) # to split the sentences
    text_sent = [sent.lower() for sent in text_sent] # to convert to lowercase
    text_sent = ["".join([char for char in text if char not in string.punctuation ]) for text in text_sent] # removed punctuation
    text_sent = [word_tokenize(sent) for sent in text_sent]
    # text_sent = ["".join([char for char in text if char not in ]) for text in text_sent] # removed punctuation
    for i in range(len(text_sent)):
        sent = " ".join(text_sent[i])
        sent=re.sub(pattern,' ',sent)
        sent=sent.replace("“"," ")
        sent=sent.replace("”"," ")
        sent=sent.replace("—"," ")
        sent=sent.replace("_"," ")
        text_sent[i]=sent.split(' ')
        text_sent[i]=[i for i in text_sent[i] if i !='']
        # print(text_sent[i])
    i=len(text_sent)-1
    while(i>=0):
        if 'chapter' in text_sent[i]:
            print(text_sent[i])
            del(text_sent[i])
        i-=1
    return text_sent
test_sent = preprocessing(data)

In [ ]:
!pip install transformers

In [52]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

In [53]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [57]:
sample = test_sent[:6]
for sent in sample:
    print(' '.join(sent))

down the rabbithole alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without pictures or conversations so she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisychain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by her
there was nothing so very remarkable in that nor did alice think it so very much out of the way to hear the rabbit say to itself oh dear
oh dear
i shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the rabbit actually took a watch out of its waistcoatpocket and looked at it and then hurried o